# Ouverture du fichier

Faites un `Ctrl+F` avec "ici" pour trouver tous les endroits où il y a un petit bug ou une optimisation à faire.  

---

## Done

Nous avons créé une série de DataFrames pour scinder un peu le projet. Voici un rappel de leur rôle :

- **`df_playlists`** : contient toutes les playlists avec leurs noms et leurs paramètres.  
  La colonne `tracks` contient toutes les informations sur les musiques de la playlist, mais dans un format très « dirty ».

- **`df_tracks`** : correspond à la colonne `playlists["tracks"]`, un peu nettoyée.  
  Une même musique peut apparaître plusieurs fois.

- **`df_tracks_vectors`** : contient `track_id` et le vecteur `list_pid`de dimension 10 000 permettant de calculer la similarité entre deux musiques.

---

## To do

- Vérifier ou optimiser le code.  
  Normalement, la question 3 est résolue.  
  Il serait intéressant de créer une fonction qui accepte **l'artiste et le titre** pour être plus user-friendly, car la fonction actuelle n'accepte que des `track_id`.  
  Il faudra préparer des exemples et tester avec potentiellement des musique que l'on connait.

- Un des "ici" correspond à un commentaire où j'essaie d'ajouter le nom de playlist dans une liste, mais cela ne fonctionne pas. Je ne comprends pas trop pourquoi, surtout que ça marche avec le `pid`. De facon dirty peut aller rechercher l'information après dans les autre df mais ce serait quand même moins optimal bien que fonctionel

- Faire des statistiques descriptives pour la question 2.

- Esquisser une formule pour la question 4
- Esquisser une formule pour la question 5


In [91]:
import pandas as pd
import numpy as np
import sys
import json
import math

In [113]:
here_Path = sys.path[0]
data_Path = here_Path + "\\challenge_set.json"
with open(data_Path, 'r', encoding='utf8') as f:
    data = json.load(f)

# Extraire uniquement la liste des playlists
playlists = data["playlists"]

# Transformer en DataFrame
df_playlists = pd.DataFrame(playlists)
df_playlists["name"] = df_playlists["name"].astype(str)
df_playlists["pid"] = df_playlists["pid"].astype(str)
df_playlists.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   name          10000 non-null  object
 1   num_holdouts  10000 non-null  int64 
 2   pid           10000 non-null  object
 3   num_tracks    10000 non-null  int64 
 4   tracks        10000 non-null  object
 5   num_samples   10000 non-null  int64 
dtypes: int64(3), object(3)
memory usage: 468.9+ KB


## On construit ici une df des tracks cleaned dans le sens ou les id sont débarrasé de *spotify:name:*
    - pid = playlist id

In [ ]:
# ici le hardcode est pas propre mais bon
all_tracks = []
df_tracks = pd.DataFrame([])
for pl in playlists:
    for tr in pl["tracks"]:
        track_info = tr.copy()  # pour ne pas modifier l'objet original
        #track_info["pname"] = pl["name"] ici je ne sais pas pourquoi mais il veut pas ajouter le nom de la playlist
        track_info["pid"] = pl["pid"]
        all_tracks.append(track_info)
df_tracks = pd.DataFrame(all_tracks)
# ici y'a moyen d'optimiser la boucle en la rentrant au dessus
for i in ( "track", "artist", "album"):
    df_tracks[i+"_id"] = df_tracks[i+"_uri"].str.split(":").str[-1]
df_tracks.drop(columns=["track_uri", "artist_uri", "album_uri"], inplace=True)
#df_tracks.head()

,pos,artist_name,track_name,duration_ms,album_name,pid,track_id,artist_id,album_id
0,0,AronChupa,Little Swing,163809,Little Swing,1000000,66U0ASk1VHZsqIkpMjKX3B,5vCOdeiQt9LyzdI87kt5Sh,4S5MLjwRSi0NJ5nikflYnZ
1,1,AronChupa,I'm an Albatraoz,166848,I'm an Albatraoz,1000000,5MhsZlmKJG6X5kTHkdwC4B,5vCOdeiQt9LyzdI87kt5Sh,1qHVYbxQ6IS8YRviorKDJI
2,2,Lorde,Yellow Flicker Beat - From The Hunger Games: M...,232506,Yellow Flicker Beat,1000000,0GZoB8h0kqXn7XFm4Sj06k,163tK9Wjr9P9DmM0AVK7lm,4UEPxQx0cTcYNsE0n32MHV
3,3,Lorde,White Teeth Teens,216600,Pure Heroine,1000000,35kahykNu00FPysz3C2euR,163tK9Wjr9P9DmM0AVK7lm,0rmhjUgoVa17LZuS8xWQ3v
4,4,Lorde,Team,193058,Pure Heroine,1000000,3G6hD9B2ZHOsgf4WfNu7X1,163tK9Wjr9P9DmM0AVK7lm,0rmhjUgoVa17LZuS8xWQ3v


A priori, si $\text{track\_id}_i = \text{track\_id}_j$ :  

$$
\text{artist\_id}_i = \text{artist\_id}_j, \quad
\text{album\_id}_i = \text{album\_id}_j
$$

On peut donc chercher par `track_id` unique et créer une liste pour le `pid`. Ainsi, *list_pid* sera un vecteur de dimension $( \text{ndim} = 10 000$. Seules les composantes non nulles sont conservées et valent 0 ou 1 si elles sont présentes dans la playlist.  
Plutôt que de conserver les 0, on conserve uniquement les indices des playlists dans lesquelles la musique apparaît.  
(On pourrait éventuellement retirer les playlists avec zéro track.)

La similarité entre deux musiques peut alors se calculer de la façon suivante :  

$$
\langle a, b \rangle = \frac{|a \cap b|}{\sqrt{\text{len}(a) \cdot \text{len}(b)}} 
$$


In [105]:
df_tracks_vectors = (
    df_tracks.groupby("track_id", as_index=True).agg({
        "pid": list
    })
)
df_tracks_vectors.rename(columns={"pid": "list_pid"}, inplace=True)


In [ ]:
def similar_tracks(track_id1,track_id2):
    try:
        list1 = df_tracks_vectors.loc[track_id1, "list_pid"]
        list2 = df_tracks_vectors.loc[track_id2, "list_pid"]
    except KeyError:
        return None
    # transformer en sets pour calculer l'intersection
    set1, set2 = set(list1), set(list2)
    n_common = len(set1 & set2)
    
    if n_common == 0:
        return 0.0
    # ici à priori pas de division par zéro car len > 0 mais bon
    return n_common / math.sqrt(len(set1) * len(set2))

# exemple d'utilisation de similar track

In [109]:
print(f"2 musiques similaires : {similar_tracks('66U0ASk1VHZsqIkpMjKX3B', '66U0ASk1VHZsqIkpMjKX3B'):.3f}")
print(f"2 musiques du meme album  : {similar_tracks('35kahykNu00FPysz3C2euR', '3G6hD9B2ZHOsgf4WfNu7X1'):.3f}")

2 musiques similaires : 1.000
2 musiques du meme album  : 0.297
